## Import Libraries

In [99]:
import importlib
import os

try:
    importlib.reload(dataloaders)
    importlib.reload(models)
except:
    import dataloaders
    import models

## Load Data

In [100]:
file = os.path.abspath("all_molecules.csv")
dataloader = dataloaders.DataLoader(file)

In [101]:
descriptors = dataloader.get_molecular_descriptors()
descriptors.shape

(2000, 10)

In [102]:
labels = dataloader.get_labels()
labels.shape

(2000, 1)

## PCA Analysis

## Model training

In [103]:
model = models.Model(descriptors, labels)

descriptors = model.preprocessing()

In [104]:
model.train()

c:\Users\marti\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('pca', PCA(n_components=0.9)),
                ('regressor', LogisticRegression())])

In [105]:
accuracy = model.test()
print(accuracy)

0.7083333333333334


In [106]:
probs = model.pipeline.predict_proba(model.X_test)[:, 0]

In [108]:
import numpy as np
print(np.sort(probs)[::-1][:100])

(600, 1)
(100,)
